# 01 Geoparsing using mordecai

Analysis reads in title and abstract data from file, runs mordecai and returns a data frame with unique locations for each entry

*Note* This code only works when running on the envname kernel, which was created from the mordecai_env_XXXXXX.yml file (details of modules also in requirements.txt).

## Set up: import modules and read in data

In [1]:
## Set up mordecai geoparser
from mordecai import Geoparser
geo = Geoparser()

2023-08-03 10:00:20.525968: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-03 10:00:20.719612: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-03 10:00:21.671367: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-08-03 10:00:21.671517: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

Models path: /home/dveytia/.local/lib/python3.8/site-packages/mordecai/models/


2023-08-03 10:00:34.542874: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2023-08-03 10:00:34.542914: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-08-03 10:00:34.543660: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# test mordecai
geo.geoparse("I traveled from Oxford to Ottawa.")

1/1 [==============================] - 0s 22ms/step


[{'word': 'Oxford',
  'spans': [{'start': 16, 'end': 22}],
  'country_predicted': 'GBR',
  'country_conf': 0.95718795,
  'geo': {'admin1': 'England',
   'lat': '51.75222',
   'lon': '-1.25596',
   'country_code3': 'GBR',
   'geonameid': '2640729',
   'place_name': 'Oxford',
   'feature_class': 'P',
   'feature_code': 'PPLA2'}},
 {'word': 'Ottawa',
  'spans': [{'start': 26, 'end': 32}],
  'country_predicted': 'CAN',
  'country_conf': 0.87992215,
  'geo': {'admin1': 'Ontario',
   'lat': '45.41117',
   'lon': '-75.69812',
   'country_code3': 'CAN',
   'geonameid': '6094817',
   'place_name': 'Ottawa',
   'feature_class': 'P',
   'feature_code': 'PPLC'}}]

In [3]:
## Import other modules
import pandas as pd
import pickle
import os
import sqlite3
from multiprocessing.pool import ThreadPool as Pool

In [7]:
## Read in test geoparsing data
#df = pd.read_csv('C:\\Users\\deviv\\Python-working-folder\\test-mordecai\\data\\raw-data\\test-geocode-text.csv')

# combine title and abstract text into one variable
#df['text'] = df['title'] + df['abstract']


(100, 6)

In [2]:
## Read in the relevance predictions
relPredCon = sqlite3.connect("\\".join(["C:","Users","deviv","Python-working-folder","test-mordecai","data","raw-data","sql-databases","relevance-predictions.sqlite"]))

relPred_df = pd.read_sql_query("SELECT * from predRel", relPredCon)

# if needed, check table names:
#cursor = relPredCon.cursor()
#cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
#print(cursor.fetchall())

# transform analysis_id column
relPred_df.analysis_id = relPred_df.analysis_id.astype(int)

relPredCon.close()
relPred_df.head()

OperationalError: unable to open database file

In [3]:
## Read in the metadata for all the unique references
uniqueRefsCon = sqlite3.connect("\\".join(["C:","Users","deviv","Python-working-folder","test-mordecai","data","raw-data","sql-databases","unique-refs_v2.sqlite"]))

## add relevance predictions as a table to the new database
#relPred_df.to_sql("predRel", uniqueRefsCon, if_exists="replace")

# print 
uniqueRefs_df = pd.read_sql_query("SELECT analysis_id, duplicate_id, title, abstract FROM uniquerefs", uniqueRefsCon)

# transform analysis_id column
uniqueRefs_df.analysis_id = uniqueRefs_df.analysis_id.astype(int)

## close connections
uniqueRefsCon.close()

uniqueRefs_df.head()

OperationalError: unable to open database file

In [27]:
uniqueRefs_df.shape

(269481, 4)

In [34]:
## subset relevance predctions to >= 0.5
relPred_df = relPred_df[0.5 <= relPred_df['relevance_mean']]
len(relPred_df)

73928

In [41]:
## JOIN the relevance predictions with the text data
df = relPred_df.set_index('analysis_id').join(uniqueRefs_df.set_index('analysis_id'), how = "left", lsuffix = "_rel", rsuffix = "_refs")
#df = relPred_df.join(uniqueRefs_df, on="analysis_id", how = "left", lsuffix = "_rel", rsuffix = "_refs")
df.head()

,relevance_mean,relevance_std,relevance_lower,relevance_upper,duplicate_id,title,abstract
analysis_id,,,,,,,
230304,0.987904,0.005010,0.982894,0.992913,2021.19382,Tidal energy site characterisation in a large ...,Banks Strait is a 15 km wide tidal channel in ...
311856,0.924882,0.172907,0.751975,1.000000,2020.20552,Integrated production system modelling and opt...,Objective/Scope: Applicability of Enhanced Oil...
354126,0.967163,0.024341,0.942821,0.991504,2011.10628,"RINA, Royal Institution of Naval Architects - ...",The proceedings contain 14 papers. The topics ...
39779,0.625630,0.303433,0.322197,0.929063,2014.2024,"Population health in the Anthropocene: Gains, ...","The health of human populations, measured by l..."
61009,0.989180,0.004243,0.984937,0.993422,2016.3747,"Seagrass, mangrove and saltmarsh sedimentary c...","Coastal vegetation (seagrass, mangrove and sal..."


In [42]:
df.shape # also get the dimensions -- for some reason I don't have matches for all the relevance predictions 36477 as opposed to 73928??

(73928, 7)

In [21]:
## Create another column with title and abstract combined to Geoparse
df['text'] = df['title'] + df['abstract']

# replace "the United States" with "United States" to avoid returning the virgin islands
# the (?i) means it is case insensitive
df['text'] = df.text.replace("(?i)the (?i)United (?i)States", "United States", regex=True) 

## alternte, but both ways work
# df['text'] = df['text'].str.replace("the United States", "United States")

## Analysis: Geoparse data and export

In [ ]:
## Geoparse the text and add results as a new column "geoparse"
#df["geoparse"] = [geo.geoparse(x) for x in df["text"].astype('str')]

In [ ]:
def geoparse_text(text):
    return geo.geoparse(text)

if __name__ = '__main__':
    with Pool() as pool:
        df["geoparse"]=pool.map(geoparse_text,df["text"].astype('str'))


In [9]:
## Format dataframe

# create new formatted dataframe: df_clean
def flat_df(df):
    df_geo = df[df["geoparse"].str.len() != 0] #subset where geoparse string is not empty
    df_geo = df_geo.explode('geoparse') #Transforms each element of a list to a row and replicates index and all other columns. When more than one place name appears it creates more than one row.
    df_geo = pd.concat([df_geo.drop(['geoparse'], axis=1), df_geo['geoparse'].apply(pd.Series)], axis=1) #Extract from dic
    df_geo = pd.concat([df_geo.drop(['geo'], axis=1), df_geo['geo'].apply(pd.Series)], axis=1)
    df_geo = df_geo[df_geo['lat'].notnull()] #Removing empty latitude rows
    df_geo.lat = df_geo.lat.astype(float) #Transforms to float
    df_geo.lon =df_geo.lon.astype(float) #Transforms to float
    return df_geo
    
df_clean = flat_df(df)
print(df_clean.dtypes)
print(df_clean[0:5])
df_clean.head(5)

Unnamed: 0             int64
analysis_id            int64
duplicate_id         float64
title                 object
abstract              object
text                  object
word                  object
spans                 object
country_predicted     object
country_conf         float32
admin1                object
lat                  float64
lon                  float64
country_code3         object
geonameid             object
place_name            object
feature_class         object
feature_code          object
0                    float64
dtype: object
   Unnamed: 0  analysis_id  duplicate_id  \
1           2        77239     2004.2448   
3           4       141394     2017.7856   
3           4       141394     2017.7856   
3           4       141394     2017.7856   
4           5       172949     2012.3571   

                                               title  \
1  Organochlorine contaminants in sea turtles: Co...   
3  New resource for population genetics studies o...   
3 

,Unnamed: 0,analysis_id,duplicate_id,title,abstract,text,word,spans,country_predicted,country_conf,admin1,lat,lon,country_code3,geonameid,place_name,feature_class,feature_code,0
1,2,77239,2004.2448,Organochlorine contaminants in sea turtles: Co...,Monitoring toxic organochlorine (OC) compounds...,Organochlorine contaminants in sea turtles: Co...,Kemp,"[{'start': 845, 'end': 849}]",USA,0.879922,Texas,32.44264,-96.22998,USA,4702914,Kemp,P,PPL,NaN
3,4,141394,2017.7856,New resource for population genetics studies o...,"The Australasian fucoid, Hormosira banksii, co...",New resource for population genetics studies o...,New South Wales,"[{'start': 856, 'end': 871}]",AUS,0.906452,New South Wales,-33.00000,146.00000,AUS,2155400,State of New South Wales,A,ADM1,NaN
3,4,141394,2017.7856,New resource for population genetics studies o...,"The Australasian fucoid, Hormosira banksii, co...",New resource for population genetics studies o...,Australia,"[{'start': 873, 'end': 882}]",AUS,0.951695,NA,-25.00000,135.00000,AUS,2077456,Commonwealth of Australia,A,PCLI,NaN
3,4,141394,2017.7856,New resource for population genetics studies o...,"The Australasian fucoid, Hormosira banksii, co...",New resource for population genetics studies o...,Hardy-Weinberg,"[{'start': 1146, 'end': 1160}]",USA,0.852964,Pennsylvania,41.40639,-75.65667,USA,7127434,Harry and Jeanette Weinberg Memorial Library,S,LIBR,NaN
4,5,172949,2012.3571,Long-term trends of coral imports into the Uni...,The international trade in corals used to be p...,Long-term trends of coral imports into the Uni...,United States,"[{'start': 43, 'end': 56}]",VIR,0.879922,NA,18.34829,-64.98348,VIR,4796775,Virgin Islands of the United States,A,PCLD,NaN


In [ ]:
## Save as a .csv
df_clean.to_csv('C:\\Users\\deviv\\Python-working-folder\\test-mordecai\\outputs\\geoparsed-records.csv', index=False)

In [ ]:
## Save as an sqlite database
database = 'C:\\Users\\deviv\\Python-working-folder\\test-mordecai\\outputs\\geoparsed-records.sqlite'
conn = sqlite3.connect(database)
df_clean.to_sql("geoparsed-records", con=conn)